### Label curation 

- First Step to initialize

In [1]:
import torch 
from collections import Counter
import random
from datasets import load_dataset
import numpy as np
import math

seed =3
random.seed(seed)
np.random.seed(seed)

dataset_name='all_train'
model_name="meta-llama/Meta-Llama-3.1-8B-Instruct"


dataset_size = 10000
confidence_prob = 0.5


all_train_dataset = load_dataset('json', data_files =f"full_dataset.json")

#################################################################################################################################
# label curation reports
report_path = f"score_curation/results/{model_name}/{dataset_name}/{dataset_name}_report.pt"




reports = torch.load(report_path)

# Part 1 (label-wise): label curation
corrupted_samples = [x[0] for x in reports.detection['label_error']]

cured_samples = []
cured_sample_labels = []
for sample in reports.curation['label_curation']:  # (idx, label, confidence)
    if sample[2] >= confidence_prob:  # confidence prob
        cured_samples.append(sample[0])
        cured_sample_labels.append((int(sample[0]), int(sample[1]), round(sample[2],2)))

print(f"Cured sample size: {len(cured_sample_labels)}")

# Filter out some cured samples from corrupted instances
cured_samples_set = set(cured_samples)
corrupted_samples_total = [x for x in corrupted_samples if x not in cured_samples_set]

print(f"Corrupted samples total: {len(corrupted_samples_total)}")

# Change the original labels to the suggested label
root_path = f"../model_finetune/selected_data/{model_name}/{dataset_name}/"


labels = torch.load(root_path + "output_labels_revised.pt")

print(f"Original Counter(labels): {Counter(labels)}")

count=0
#identify the transition labels
count_labels_5 = []
count_labels_4 = []
count_labels_3 = []
count_labels_2 = []

for sample_label in cured_sample_labels:
    if labels[sample_label[0]] == 5:
        count_labels_5.append(sample_label)
        # continue ## determine whether remain the 5-rated samples

    if labels[sample_label[0]] == 4:
        count_labels_4.append(sample_label)

    if labels[sample_label[0]] == 3:
        count_labels_3.append(sample_label)

    if labels[sample_label[0]] == 2:
        count_labels_2.append(sample_label)

    labels[sample_label[0]] = sample_label[1]
    count+=1

print(f"counting revised label size: {count}")

print(f"Label size: {len(labels)}")
label_counts = Counter(labels)

print(f"Revised Counter(labels): {label_counts}")

# Filter out the low-quality samples
label_wise_filter_out_samples = set(corrupted_samples_total)
print(f"Label-wise filter out samples: {len(label_wise_filter_out_samples)}")

### load the label noise 
# torch.save(labels, root_path + f"output_labels_revised_cured.pt")


/home/jlpang/Docta/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1538163/3981394206.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weig

==== Docta: Doctor for your data. Current version: 0.2 ====
Cured sample size: 10510
Corrupted samples total: 112457
Original Counter(labels): Counter({3: 87975, 2: 86132, 4: 59969, 1: 44401, 0: 18626, 5: 3829})
counting revised label size: 10510
Label size: 300932
Revised Counter(labels): Counter({3: 89665, 2: 86085, 4: 61057, 1: 43117, 0: 18503, 5: 2505})
Label-wise filter out samples: 112457


/tmp/ipykernel_1538163/3981394206.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  labels = torch.load(root_path + "output_labels_revised.pt")


## Baseline: Label-filtered algorithm 

- tag: label-filtered

In [ ]:
import torch 
from collections import Counter
import random
from datasets import load_dataset
import numpy as np

seed = 3
random.seed(seed)
np.random.seed(seed)


label_filtered_indices = []
for target_label in [5, 4, 3, 2, 1]:
    if len(label_filtered_indices) == dataset_size:
        break   

    indices = [i for i, label in enumerate(labels) if label == target_label]

    if dataset_size - len(label_filtered_indices) > len(indices):
        label_filtered_indices.extend(indices)
    else:
        random_indices = np.random.permutation(len(indices))[:dataset_size-len(label_filtered_indices)]
        label_filtered_indices.extend(random_indices)

print(f"data size: {len(label_filtered_indices)}")

label_filtered_dataset = all_train_dataset['train'].select(label_filtered_indices)

label_filtered_labels = np.array(labels)[label_filtered_indices].tolist()

root_path = f"./model_finetune/selected_data/{model_name}/{dataset_name}/"

label_filtered_dataset.to_json(root_path + f"label-filtered-cured-{confidence_prob}_dataset.json")


data size: 10000


Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 38.40ba/s]


16755007

## Baseline: diversity-filtered

- tag: diversity-filtered

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset
import random
import numpy as np
from tqdm import tqdm
import os

seed = 3
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def cosDistance(sample_embedding, selected_embeddings, k_near=10):
    sample_embedding = sample_embedding.to(device)
    selected_embeddings = selected_embeddings.to(device)

    similarity_vector = torch.matmul(selected_embeddings, sample_embedding)
    distance_vector = 1.0 - similarity_vector

    if selected_embeddings.size(0) > k_near:
        distance_vector, _ = torch.topk(distance_vector, k=k_near, dim=0)
    
    mean_distance = distance_vector.mean().item()  #
    return mean_distance

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def process_dialog(dialog):
    conversation = ""
    for message in dialog['messages']:
        conversation += f"### {message['role']}: {message['content']}\n"
    return {"features": conversation}

def embed_text(batch):
    encoded_inputs = tokenizer(batch['features'], padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        model_outputs = model(**encoded_inputs)
    sentence_embeddings = mean_pooling(model_outputs, encoded_inputs['attention_mask'])
    embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    
    batch['embeddings'] = embeddings.cpu().numpy().tolist()  
    return batch

##############################################################################################################################

threshold = 0.5
k_near = 10

if not os.path.exists(f"{dataset_name}_embeddings.parquet"):
    data = load_dataset('json', data_files=f"./data/train_data/{dataset_name}_data.jsonl")
    data['train'] = data['train'].map(process_dialog, batched=False)

    embedding_model_name = "BAAI/bge-large-en-v1.5"
    tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
    model = AutoModel.from_pretrained(embedding_model_name).to(device)
    model = torch.nn.DataParallel(model)

    data['train'] = data['train'].map(embed_text, batched=True, batch_size=2048)
    data['train'].to_parquet(f'{dataset_name}_embeddings.parquet')
    print(f"Embeddings saved to {dataset_name}_embeddings.parquet")

#########################################################################################################################

label_priority = [5, 4, 3, 2, 1]
label_indices = {label: [idx for idx, lbl in enumerate(labels) if lbl == label] for label in label_priority}

embedding_dataset = load_dataset('parquet', data_files=f'{dataset_name}_embeddings.parquet')['train']

selected_embeddings = None
selected_samples = []

for label in label_priority:
    if len(selected_samples) >= dataset_size:
        break
    
    embedding_subset = embedding_dataset.select(label_indices[label])
    
    for sample in tqdm(embedding_subset, desc=f"Processing high-quality samples (label={label})"):
        sample_embedding = torch.tensor(sample['embeddings']).to(device)  
        
        if selected_embeddings is None:
            selected_embeddings = sample_embedding.unsqueeze(0)
            selected_samples.append(sample)
            continue
        
        if cosDistance(sample_embedding, selected_embeddings, k_near=k_near) < threshold:
            selected_embeddings = torch.cat((selected_embeddings, sample_embedding.unsqueeze(0)), dim=0)
            selected_samples.append(sample)
        
        if len(selected_samples) >= dataset_size:
            break
    print(f"Selected data size from label {label}: {len(selected_samples)}")

selected_dataset = Dataset.from_dict({col: [s[col] for s in selected_samples] for col in selected_samples[0].keys()})
print(f"Selected data size: {len(selected_dataset)}")

selected_dataset.to_json(root_path + f'diversity-filtered-cured-{confidence_prob}_dataset.json', orient='records', lines=True)


Using device: cuda


Processing high-quality samples (label=5): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 487/487 [00:06<00:00, 73.24it/s]


Selected data size from label 5: 107


Processing high-quality samples (label=4): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66181/66181 [01:47<00:00, 617.09it/s]


Selected data size from label 4: 8085


Processing high-quality samples (label=3):   5%|██████████████████████████████████████████                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 5460/118313 [00:04<01:40, 1125.26it/s]


Selected data size from label 3: 10000
Selected data size: 10000


Creating json from Arrow format: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.72ba/s]


177782124

## Filetered method: Our Data selection method
- Filtered 5: label-filtered based: all 5 samples  + 4-rated samples select using sorted long-tail score (reverse=True)

In [ ]:
import torch
import random
import numpy as np
from datasets import load_dataset
from collections import Counter

# Set random seed for reproducibility
seed = 3
random.seed(seed)
np.random.seed(seed)

# Part 2 (feature-wise): Process rare samples based on 'rare_example' detection
rare_samples = reports.detection['rare_example'][:len(reports.detection['rare_example']) // 2]
rare_samples_filtered = np.array(rare_samples)[:, :2]  # Use NumPy for faster operations

print(f"Size of the remaining samples with high quality: {len(rare_samples_filtered)}")

# Assume 'labels' is a Python list; convert it to a NumPy array for efficient indexing
labels = np.array(labels)

# Cache label indices to avoid repeated searches
label_indices_cache = {label: np.where(labels == label)[0] for label in [5, 4, 3, 2, 1]}
print(f"Finished caching labels indices...")

# Initialize list to store selected indices
filtered_indices = []

# Filter and sort samples by label
for target_label in [5, 4, 3, 2, 1]:
    if len(filtered_indices) >= dataset_size:
        break

    # Get indices of current label
    label_indices = label_indices_cache[target_label]
    available_size = dataset_size - len(filtered_indices)

    # Add label indices if enough space, else sort and add top samples
    if available_size > len(label_indices):
        filtered_indices.extend(label_indices.tolist())
    else:
        # Filter and sort samples with the target label by score
        label_samples = rare_samples_filtered[np.isin(rare_samples_filtered[:, 0], label_indices)]
        if len(label_samples) > 0:  # Ensure label_samples is not empty
            sorted_samples = label_samples[label_samples[:, 1].argsort()[::-1]][:available_size]
            filtered_indices.extend(sorted_samples[:, 0].astype(int).tolist())

    print("Size of the filtered dataset:", len(filtered_indices))

# Load the dataset and filter out samples by selected indices
data = load_dataset('json', data_files=root_path + 'full_dataset.json')

# Select and save filtered samples
filtered_dialogs = data['train'].select(filtered_indices)
filtered_dialogs.to_json(root_path + f"filtered-cured-{confidence_prob}_dataset.json")
print(f"Filtered dataset saved to {root_path}filtered-cured-{confidence_prob}_dataset.json")
